In [2]:
users = '{"oliver.a.simmonds@gmail.com": "dev_team", "abdi212121@gmail.com": "dev_team", "andygregory17@hotmail.com": "golf_breaks", "agregory@golfbreaks.com": "golf_breaks"}'
import json
users = json.loads(users)
print(users)



{'oliver.a.simmonds@gmail.com': 'dev_team', 'abdi212121@gmail.com': 'dev_team', 'andygregory17@hotmail.com': 'golf_breaks', 'agregory@golfbreaks.com': 'golf_breaks'}


In [1]:
from src.ai.web_scraper import crawl_website

scraped_data = crawl_website("https://www.golfbreaks.com/en-gb/", max_depth=int(1), time_limit=20)


2025-04-08 21:24:17.536 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 21:24:18.010 
  command:

    streamlit run c:\Users\olive\anaconda3\envs\veloflow\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-08 21:24:18.011 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 21:24:23.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 21:24:23.684 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
scraped_data

{'https://www.golfbreaks.com/en-gb/': "\n\t\tGet first-hand reviews and the latest travel trends with the Golf Travel Experts.\n\t\n \n\t\t\t\t\t\t\t\tOpen tomorrow 8am - 7pm\n\n\t\t\t\t\t\t\t01753 720 321\n\t\t\t\t\t\t\n \n\t\t\t\t\t\t\t\tOpen tomorrow 8am - 7pm\n\n\t\t\t\t\t\t\t01753 720 321\n\t\t\t\t\t\t\n \n\t\t\t\t\t\t\t\tOpen tomorrow 8am - 7pm\n\n\t\t\t\t\t\t\t01753 720 321\n\t\t\t\t\t\t\n \n\t\t\t\t\t\t\t\tOpen tomorrow 8am - 7pm\n\n\t\t\t\t\t\t\t01753 720 321\n\t\t\t\t\t\t\n \n\t\t\t\t\t\t\t\tOpen tomorrow 8am - 7pm\n\n\t\t\t\t\t\t\t01753 720 321\n\t\t\t\t\t\t\n \n\t\t\t\t\t\t\t\tOpen tomorrow 8am - 7pm\n\n\t\t\t\t\t\t\t01753 720 321\n\t\t\t\t\t\t\n \n\t\t\t\t\t\t\t\tOpen tomorrow 8am - 7pm\n\n\t\t\t\t\t\t\t01753 720 321\n\t\t\t\t\t\t\n \n\t\t\t\t\t\t\t\tOpen tomorrow 8am - 7pm\n\n\t\t\t\t\t\t\t01753 720 321\n\t\t\t\t\t\t\nAs the world's #1 rated golf tour operator, and with over 25 years’ experience as the market leader in global golf travel, enjoy peace of mind when booking 

In [3]:


extract_unique_sentences(scraped_data)

NameError: name 'extract_unique_sentences' is not defined

In [12]:
s = ' '.join([item for sublist in extract_unique_sentences(scraped_data).values() for item in sublist])
print((s))

Open today 8am - 7pm

							01753 720 321
						
 
								Open today 8am - 7pm

							01753 720 321
						
 
								Open today 8am - 7pm

							01753 720 321
						
 
								Open today 8am - 7pm

							01753 720 321
						
 
								Open today 8am - 7pm

							01753 720 321
						
 
								Open today 8am - 7pm

							01753 720 321
						
 
								Open today 8am - 7pm

							01753 720 321
						
 
								Open today 8am - 7pm

							01753 720 321
						
As the world's #1 rated golf tour operator, and with over 25 years’ experience as the market leader in global golf travel, enjoy peace of mind when booking your next golf holiday or break. Looking for your next bucket-list golf holiday? Fear not, with over 2,000 venues and destinations worldwide, you’re never short of remarkable golf experiences. Whether that's playing bucket-list courses in Scotland, experiencing incredible value in Spain, or indulging in all-inclusive bliss in the Caribbean, we offer extensive choice and e

In [ ]:
import streamlit as st
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import re
from collections import Counter
import concurrent.futures

def scrape_page(url, headers):
    """
    Scrapes the content of a single page.
    Returns:
      - The concatenated text from all <p> tags.
      - A list of hyperlinks found on the page.
    """
    try:
        response = requests.get(url, headers=headers)
    except Exception as e:
        st.error(f"Error accessing {url}: {e}")
        return "", []
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract text from paragraphs
        paragraphs = soup.find_all('p')
        content = "\n".join(p.get_text() for p in paragraphs)
        # Extract all 'a' tag hyperlinks
        raw_links = [a.get('href') for a in soup.find_all('a', href=True)]
        # Convert relative links to absolute
        links = [urljoin(url, link) for link in raw_links]
        return content, links
    else:
        st.warning(f"Failed to retrieve content from {url}. HTTP Status Code: {response.status_code}")
        return "", []

def crawl_website(start_url, max_depth=2, delay=0.1, time_limit=20):
    """
    Crawls a website starting from start_url concurrently.
    
    Parameters:
      start_url (str): The URL of the website to start crawling from.
      max_depth (int): Maximum recursion depth to prevent endless crawling.
      delay (float): A small delay between requests (not used in this version).
      time_limit (int): Maximum time in seconds to let the crawl run.
    
    Returns:
      dict: A mapping from each visited URL to its scraped content.
    """
    headers = {
        'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                       'AppleWebKit/537.36 (KHTML, like Gecko) '
                       'Chrome/58.0.3029.110 Safari/537.36')
    }
    
    visited = set()       # Keep track of visited URLs to avoid duplication
    results = {}          # Dictionary to store the scraped content by URL
    base_domain = urlparse(start_url).netloc
    start_time = time.time()
    
    # Dictionary to map futures to their (url, depth)
    future_to_task = {}
    
    # Create a thread pool executor for concurrent requests
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=8)
    
    def schedule_crawl(url, depth):
        if (time.time() - start_time) > time_limit:
            return
        if depth > max_depth or url in visited:
            return
        visited.add(url)
        # Schedule a concurrent fetch of the page
        future = executor.submit(scrape_page, url, headers)
        future_to_task[future] = (url, depth)
    
    # Begin crawling from the start_url
    schedule_crawl(start_url, 0)
    
    # Process futures as they complete and schedule new tasks
    while future_to_task and (time.time() - start_time) < time_limit:
        # Wait until any of the scheduled tasks is completed
        done, _ = concurrent.futures.wait(future_to_task, return_when=concurrent.futures.FIRST_COMPLETED)
        for future in done:
            url, depth = future_to_task.pop(future)
            try:
                content, links = future.result()
                results[url] = content
                # For each link found, schedule a crawl if it's internal
                for link in links:
                    parsed_link = urlparse(link)
                    if parsed_link.netloc and parsed_link.netloc != base_domain:
                        continue
                    if link not in visited:
                        schedule_crawl(link, depth + 1)
            except Exception as e:
                st.error(f"Error processing {url}: {e}")
    
    executor.shutdown(wait=True)
    return results

def extract_unique_sentences(pages_dict):
    """
    Splits all text into sentences, counts them and returns only unique ones.
    """
    sentence_counter = Counter()
    page_sentences = {}

    for url, content in pages_dict.items():
        sentences = re.split(r'(?<=[.!?])\s+', content.strip())
        page_sentences[url] = sentences
        sentence_counter.update(sentences)

    unique_sentences = {}
    for url, sentences in page_sentences.items():
        unique = [s for s in sentences if sentence_counter[s] == 1]
        unique_sentences[url] = unique

    return ' '.join([sentence for sentences in unique_sentences.values() for sentence in sentences])

start_url = "https://www.golfbreaks.com/en-gb/"
pages = crawl_website(start_url)
unique_text = extract_unique_sentences(pages)
st.write("Extracted unique sentences:")
st.write(unique_text)


2025-04-08 21:40:32.576 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'ThreadPoolExecutor-3_3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 21:40:33.103 
  command:

    streamlit run c:\Users\olive\anaconda3\envs\veloflow\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-08 21:40:33.105 Thread 'ThreadPoolExecutor-3_3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 21:40:33.935 Thread 'ThreadPoolExecutor-3_6': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 21:40:33.937 Thread 'ThreadPoolExecutor-3_6': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 21:41:32.008 Thread 'ThreadPoolExecutor-3_6': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 21:41:32.009 Thread 'ThreadPoolExecutor-3_6': missing ScriptRunContext! This warning can

In [16]:
import streamlit as st
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import re
from collections import Counter
import concurrent.futures

def scrape_page(url, headers):
    """
    Scrapes the content of a single page.
    Returns:
      - The concatenated text from all <p> tags.
      - A list of hyperlinks found on the page.
    """
    try:
        response = requests.get(url, headers=headers)
    except Exception as e:
        st.error(f"Error accessing {url}: {e}")
        return "", []
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        content = "\n".join(p.get_text() for p in paragraphs)
        raw_links = [a.get('href') for a in soup.find_all('a', href=True)]
        links = [urljoin(url, link) for link in raw_links if link]
        return content, links
    else:
        st.warning(f"Failed to retrieve content from {url}. HTTP Status Code: {response.status_code}")
        return "", []

def crawl_website(start_url, max_depth=2, time_limit=20):
    """
    Crawls a website starting from start_url concurrently.
    
    Parameters:
      start_url (str): The URL of the website to start crawling from.
      max_depth (int): Maximum recursion depth to prevent endless crawling.
      time_limit (int): Maximum time in seconds to let the crawl run.
    
    Returns:
      dict: A mapping from each visited URL to its scraped content.
    """
    headers = {
        'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                       'AppleWebKit/537.36 (KHTML, like Gecko) '
                       'Chrome/58.0.3029.110 Safari/537.36')
    }
    
    visited = set()      # Keep track of visited URLs to avoid duplication
    results = {}         # Dictionary to store the scraped content by URL
    base_domain = urlparse(start_url).netloc
    start_time = time.time()
    
    # Dictionary to map futures to their (url, depth)
    future_to_task = {}
    
    # Create a thread pool executor for concurrent requests
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=8)
    
    def schedule_crawl(url, depth):
        if time.time() - start_time > time_limit:
            return
        if depth > max_depth or url in visited:
            return
        visited.add(url)
        future = executor.submit(scrape_page, url, headers)
        future_to_task[future] = (url, depth)
    
    # Begin crawling from the start_url
    schedule_crawl(start_url, 0)
    
    # Process futures as they complete and schedule new tasks
    while future_to_task and (time.time() - start_time) < time_limit:
        # Wait until any of the scheduled tasks is completed
        done, _ = concurrent.futures.wait(future_to_task, return_when=concurrent.futures.FIRST_COMPLETED, timeout=0.5)
        for future in done:
            url, depth = future_to_task.pop(future)
            try:
                content, links = future.result()
                results[url] = content
                for link in links:
                    parsed_link = urlparse(link)
                    if parsed_link.netloc and parsed_link.netloc != base_domain:
                        continue
                    if link not in visited:
                        schedule_crawl(link, depth + 1)
            except Exception as e:
                st.error(f"Error processing {url}: {e}")
    
    # Cancel any futures still pending once the time limit is reached
    for future in future_to_task:
        future.cancel()
    
    executor.shutdown(wait=False)
    return results

import re
from collections import Counter
import unicodedata

def extract_unique_sentences(pages_dict):
    """
    Splits all text into sentences, preprocesses them,
    counts their frequency, and returns only truly unique ones (preprocessed).
    """
    sentence_counter = Counter()
    page_sentences = {}
    sentence_mapping = {}  # Maps original sentence -> preprocessed form

    def preprocess(sentence):
        # Normalize unicode, lowercase, strip punctuation and extra spaces
        sentence = unicodedata.normalize("NFKD", sentence)
        sentence = sentence.lower().strip()
        sentence = re.sub(r'[\s]+', ' ', sentence)
        # sentence = re.sub(r'[^\w\s]', '', sentence)  # remove punctuation
        return sentence

    # First pass: split and preprocess
    for url, content in pages_dict.items():
        sentences = re.split(r'(?<=[.!?])\s+', content.strip())
        page_sentences[url] = sentences
        for s in sentences:
            p = preprocess(s)
            sentence_counter[p] += 1
            sentence_mapping[s] = p

    # Second pass: collect preprocessed sentences that are unique
    unique_sentences = set()
    for sentences in page_sentences.values():
        for s in sentences:
            preprocessed = sentence_mapping[s]
            if sentence_counter[preprocessed] == 1:
                unique_sentences.add(preprocessed)

    return ' '.join(unique_sentences)

# Example usage:
start_url = "https://www.golfbreaks.com/en-gb/"
pages = crawl_website(start_url)
print(pages.keys())
unique_text = extract_unique_sentences(pages)
# print(unique_text)


2025-04-08 21:59:02.143 Thread 'ThreadPoolExecutor-5_3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 21:59:02.150 Thread 'ThreadPoolExecutor-5_3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 21:59:03.410 Thread 'ThreadPoolExecutor-5_0': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 21:59:03.413 Thread 'ThreadPoolExecutor-5_0': missing ScriptRunContext! This warning can be ignored when running in bare mode.


dict_keys(['https://www.golfbreaks.com/en-gb/', 'tel:01753 752 900', 'https://www.golfbreaks.com/en-gb/rewards/', 'https://www.golfbreaks.com/en-gb/offers/best-selling/uk-ireland/', 'https://www.golfbreaks.com/en-gb/brochure/', 'https://www.golfbreaks.com/en-gb/inspiration/', 'https://www.golfbreaks.com/en-gb/inspiration/best-golf-courses/', 'https://www.golfbreaks.com/en-gb/inspiration/tips-trends/', 'https://www.golfbreaks.com/en-gb/collections/golf-holidays/', 'tel:01753 720 321', 'https://www.golfbreaks.com/en-gb/inspiration/best-golf-resorts/', 'https://www.golfbreaks.com/en-gb/holidays/gloucestershire/bells-hotel-forest-of-dean-golf-club/packages/5a27a5e2-6bdf-4d33-8231-a72e35224e96/?date=2025-04-27', 'https://www.golfbreaks.com/en-gb/holidays/belek/sueno-hotels-deluxe-belek/packages/a8f665bc-76b8-4d25-90f1-29ab588df105/?date=2025-11-23', 'https://www.golfbreaks.com/en-gb/tournaments/the-masters/', 'https://www.golfbreaks.com/en-gb/offers/last-minute/uk-and-ireland/', 'https://ww

In [17]:
unique_text

'if you’re restricted by specific months and/or seasons of the year, then fear not, as we offer awe-inspiring locations around the world, throughout the year. browse our cheap golf holidays in portugal here and enjoy the warm climate with 300 days of glorious sunshine a year. on the whole, our golfers generally opt for either marrakech or agadir, with five-star hotels aplenty in both cities including fairmont royal palm marrakech, kenzi menara palace and movenpick hotel mansour eddehbi. haven\'t found the perfect deal yet? royal park i roveri, turin for those in search of an easy, relaxing, yet luxurious golf holiday, italy has plenty to offer. scotland is a golfing paradise, steeped in history and tradition. the algarve offers over 300 days of sunshine per year, with a host of stay & play golf resorts and the lively towns of vilamoura and albufeira – perfect for groups looking for a good night out. golf & nightlife when on a moroccan golf holiday, their are several places to visit to 

In [18]:
print(len(unique_text)/5)

111884.0
